![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")
ds_jobs

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,city_173,0.878,Male,No relevant experience,no_enrollment,Graduate,Humanities,14,NaN,NaN,1,42,1.0
19154,31398,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,14,NaN,NaN,4,52,1.0
19155,24576,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,44,0.0
19156,5756,city_65,0.802,Male,Has relevant experience,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,97,0.0


In [2]:
# Create a copy of ds_jobs for transforming
df = ds_jobs.copy()

# take a look at the dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [3]:
# take a look at the memory each column use
df.memory_usage()

Index                        128
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64

In [4]:
# EDA to help identify ordinal, nominal, and two-factor categories
for col in ds_jobs.select_dtypes("object").columns:
    print(df[col].value_counts(), '\n')

city_103    4355
city_21     2702
city_16     1533
city_114    1336
city_160     845
            ... 
city_129       3
city_111       3
city_121       3
city_140       1
city_171       1
Name: city, Length: 123, dtype: int64 

Male      13221
Female     1238
Other       191
Name: gender, dtype: int64 

Has relevant experience    13792
No relevant experience      5366
Name: relevant_experience, dtype: int64 

no_enrollment       13817
Full time course     3757
Part time course     1198
Name: enrolled_university, dtype: int64 

Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: education_level, dtype: int64 

STEM               14492
Humanities           669
Other                381
Business Degree      327
Arts                 253
No Major             223
Name: major_discipline, dtype: int64 

>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
1

# Columns containing categories with only two factors must be stored as Booleans (bool).

In [5]:
# check for columns that have only two factors
for x in df.columns:
    print(f"number of unique values in column {x} is: {df[x].nunique()}")
    
# columns ["relevant_experience", "job_change"]

number of unique values in column student_id is: 19158
number of unique values in column city is: 123
number of unique values in column city_development_index is: 93
number of unique values in column gender is: 3
number of unique values in column relevant_experience is: 2
number of unique values in column enrolled_university is: 3
number of unique values in column education_level is: 5
number of unique values in column major_discipline is: 6
number of unique values in column experience is: 22
number of unique values in column company_size is: 8
number of unique values in column company_type is: 6
number of unique values in column last_new_job is: 6
number of unique values in column training_hours is: 241
number of unique values in column job_change is: 2


In [6]:
rel_ex = {"Has relevant experience" : True, "No relevant experience" : False}
df["relevant_experience"] = df["relevant_experience"].replace(rel_ex) 

job_change = {1.0: True, 0.0: False}
df["job_change"] = df["job_change"].replace(job_change)

df[["relevant_experience", "job_change"]].head()

,relevant_experience,job_change
0,True,True
1,False,False
2,False,False
3,False,True
4,True,False


# Columns containing integers only must be stored as 32-bit integers (int32).

In [7]:
# check for columns containing integers only
for x in df.columns:
    print(f"the datatype foe the column {x} is: {df[x].dtype}")

# integer ["student_id", "training_hours"]
# float ["city_development_index"]
# nominal categorical (without order) ["city", "gender", "major_discipline", "company_type"]
# ordinal categorical data (with order) ["education_level", "experience", "company_size", "last_new_job"]

the datatype foe the column student_id is: int64
the datatype foe the column city is: object
the datatype foe the column city_development_index is: float64
the datatype foe the column gender is: object
the datatype foe the column relevant_experience is: bool
the datatype foe the column enrolled_university is: object
the datatype foe the column education_level is: object
the datatype foe the column major_discipline is: object
the datatype foe the column experience is: object
the datatype foe the column company_size is: object
the datatype foe the column company_type is: object
the datatype foe the column last_new_job is: object
the datatype foe the column training_hours is: int64
the datatype foe the column job_change is: bool


In [8]:
# integer ["student_id", "training_hours"]

df["student_id"] = df["student_id"].astype(np.int32)

df["training_hours"] = df["training_hours"].astype(np.int32)

print(df["student_id"].dtype, df["training_hours"].dtype)

int32 int32


# Columns containing floats must be stored as 16-bit floats (float16).

In [9]:
# float ["city_development_index"]

df["city_development_index"] = df["city_development_index"].astype(np.float16)

print(df["city_development_index"].dtype)

float16


# Columns containing nominal categorical data must be stored as the category data type.

In [10]:
# Columns containing nominal categorical data (means that does not have an order)
# nominal categorical (without order) ["city", "gender", "enrolled_university", "major_discipline", "company_type"]

nominal_cols = ["city", "gender", "major_discipline", "company_type"]
for col in nominal_cols: 
    df[col] = df[col].astype("category")
    print(f"the column {col} datatype is: {df[col].dtype}")

the column city datatype is: category
the column gender datatype is: category
the column major_discipline datatype is: category
the column company_type datatype is: category


# Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.

In [11]:
# Create a dictionary of columns containing ordered categorical data
ordered_cats = {
    'enrolled_university': ['no_enrollment', 'Part time course', 'Full time course'],
    'education_level': ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'],
    'experience': ['<1'] + list(map(str, range(1, 21))) + ['>20'],
    'company_size': ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
    'last_new_job': ['never', '1', '2', '3', '4', '>4']
}

for col in ordered_cats.keys():
    category = pd.CategoricalDtype(ordered_cats[col], ordered=True)
    df[col] = df[col].astype(category)

# The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.

In [12]:
df = df[(df["experience"] >= "10") & (df["company_size"] >= "1000-4999")]
ds_jobs_transformed = df
ds_jobs_transformed.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,NaN,True,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,False
12,25619,city_61,0.913086,Male,True,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,False
31,22293,city_103,0.919922,Male,True,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,False
34,26494,city_16,0.910156,Male,True,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,False
40,2547,city_114,0.925781,Female,True,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,False


In [13]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   bool    
 5   enrolled_university     2185 non-null   category
 6   education_level         2184 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            2184 non-null   category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   bool    
dtypes: bool(2), category(9)

In [14]:
ds_jobs_transformed.memory_usage()

Index                     17608
student_id                 8804
city                       7353
city_development_index     4402
gender                     2333
relevant_experience        2201
enrolled_university        2333
education_level            2413
major_discipline           2421
experience                 2933
company_size               2565
company_type               2421
last_new_job               2421
training_hours             8804
job_change                 2201
dtype: int64

## now we can notice a substantial decrease in memory usage which will make the modeling much faster and more efficient